In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, time

In [2]:
# Station and wharves dataframe
wharf_df = pd.read_csv('csv_inputs/wharf_info.csv')
# lines dataframe
line_df = pd.read_csv('csv_inputs/line_info.csv')
line_df['First_sailing'] = pd.to_datetime(line_df['First_sailing'], format='%H:%M')
# Wharf to wharf transit time dataframe
tt_df = pd.read_csv('csv_inputs/rebalancing_times.csv',index_col='From/To')
# Headways dataframe
headway_df = pd.read_csv('csv_inputs/headways.csv')
# vessels
vessel_df = pd.read_csv('csv_inputs/vessel_info.csv')
# charging berths dataframe
charging_berth = pd.read_csv('csv_inputs/charging_berths.csv')

# Simulation time parameters
initial_time = time(5,30)
period_length = 5  # mins
total_operation_hours = 6  # hours

# nc, Minimum num of crew breaks
nc = 1

# Dc, Crew break duration (fixed)
Dc = 9  # mins

# Tc, Maximum separation time for crew breaks
Tc = 6 * 60  # mins

# rv+, charging rate
rv_plus = 2100 * period_length / 60 / 1100  # kW*h/kWh --> %

# pc, Plugging/Unplugging time
pc = 2  # mins



# Functions

In [4]:
def cal_time(period_num):
    """
    Calculates a new datetime based on an initial time and a given number of periods.
    
    Args:
    period_num (int): The period number to calculate the time for.
    initial_time (datetime.time): The starting time of the calculation.
    period_length (int): The length of each period in minutes.
    
    Returns:
    datetime.datetime: The calculated datetime.
    """
    # Convert initial_time to a datetime object with today's date
    initial_datetime = datetime.combine(datetime.today(), initial_time)
    
    # Calculate the total minutes to add
    added_time = timedelta(minutes=int((period_num - 1) * period_length))
    new_time = initial_datetime + added_time

    return new_time


In [21]:
def cal_duration(minutes):
    result = (minutes // 5) + (1 if minutes % 5 != 0 else 0)
    return int(result)

def cal_h(s, d, l): # headways
    try:
        h = headway_df[f'h{l}'].dropna().tolist()
        h_sd_ls = [d]  # Start the list with the initial day 'd'
        for sailing_headway in h:
            num_time_period = cal_duration(sailing_headway)
            h_sd_ls.append(h_sd_ls[-1] + num_time_period)
        if s-1 < len(h_sd_ls):
            return h_sd_ls[s-1]
        else:
            return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [35]:
# Functions to replace and determine wharfs
# def start_wharf(task):
#     if task.isdigit():
#         return Start_wharf[int(task)]
#     return task
def start_wharf(task):
    # Ensure task is a string key for the dictionary lookup
    if task.isdigit():
        task_str = task  # Keep task as string to match dictionary key
        if task_str in Start_wharf:
            return Start_wharf[task_str]
        else:
            # Handle missing key case
            print(f"Key {task_str} not found in Start_wharf, using default.")
            return 'Unknown Wharf'
    return task

def end_wharf(row):
    wharf = row['End_Wharf']
    if row['Task'].isdigit():
        line = int(row['Task'])
        timetoT = int(line_df[line_df['Line_No'] == line]['Time_underway_to_T'].iloc()[0])
        end_time = row['Start_Time'] + timetoT // period_length + 1
        matching_rows = Zp_df[(Zp_df['Line'] == line) & (Zp_df['Time'] == end_time)]
        if not matching_rows.empty:
            wharf = matching_rows['Wharf'].iloc()[0]
    return wharf

def end_time(row):
    if row['Task'].isdigit():
        line = int(row['Task'])
        duration = int(line_df[line_df['Line_No'] == line]['Line_duration'].iloc()[0])
        return (cal_time(row['Start_Time']) + timedelta(minutes=duration)).strftime('%H:%M')
    elif row['Task'] in ['Waiting', 'Charging']:
        return (cal_time(row['Start_Time']) + timedelta(minutes=5)).strftime('%H:%M')
    else:
        return (cal_time(row['Start_Time']) + timedelta(minutes=Dc)).strftime('%H:%M')

# Load results files

In [29]:
def load_and_process_data(filepath, split_columns, value_columns):
    """
    Load data, remove unwanted characters, filter rows, split columns, and convert types.

    Args:
    filepath (str): Path to the CSV file.
    split_columns (list): List of columns names after splitting the 'Variable' column.
    value_columns (dict): Dictionary of columns to convert with their respective types.
    """
    df = pd.read_csv(filepath)
    df = df[df['Value'] == 1]
    # df['Variable'] = df['Variable'].str.replace(r"[()']", "", regex=True)
    df['Variable'] = df['Variable'].str.replace(r"[()' ]", "", regex=True)
    df[split_columns] = df['Variable'].str.split(',', expand=True)
    for column, dtype in value_columns.items():
        df[column] = df[column].astype(dtype)
    return df


# # Load and process each DataFrame using the new function
# z_df = load_and_process_data('output_files/z_wj_results.csv',['Wharf', 'Line'],{'Line': int})

# Zp_df = load_and_process_data('output_files/Z_prime_lwt_results.csv',['Line', 'Wharf', 'Time'],{'Line': int, 'Time': int})

# Z_df = load_and_process_data('output_files/Z_lwt_results.csv',['Line', 'Wharf', 'Time'],{'Line': int, 'Time': int})

file_prefix = "6htest_cyclelines_rob_sol"

z_df = load_and_process_data(f'output_files/{file_prefix}_z_wj_results.csv',['Wharf', 'Task'],{})

Zp_df = load_and_process_data(f'output_files/{file_prefix}_Z_prime_lwt_results.csv',['Line', 'Wharf', 'Time'],{'Line': int, 'Time': int})

Z_df = load_and_process_data(f'output_files/{file_prefix}_Z_lwt_results.csv',['Line', 'Wharf', 'Time'],{'Line': int, 'Time': int})

Start_S = dict(zip(line_df['Line_No'].astype(str), line_df['O']))
Start_S.update(dict(zip(wharf_df['Wharf_No'], wharf_df['Station'])))
End_S = dict(zip(line_df['Line_No'].astype(str), line_df['T']))
End_S.update(dict(zip(wharf_df['Wharf_No'], wharf_df['Station'])))

# Start_wharf = dict(zip(z_df['Line'].astype(str), z_df['Wharf']))
linels = z_df['Task'].unique().tolist()

Start_wharf = {}
for line in linels:
    Start_wharf[line] = z_df[z_df['Task'] == str(line)]['Wharf'].iloc[0]

# Timetable

In [24]:
# Load and preprocess x_df
# x_df = load_and_process_data('output_files/x_ld_results.csv',['Line', 'Time'],{'Line': int, 'Time': int})
x_df = load_and_process_data(f'output_files/{file_prefix}_x_ld_results.csv',['Line', 'Time'],{'Line': int, 'Time': int})


# List of lines from line_df and x_df
lines = line_df['Line_No'].unique().tolist()
solved_lines = x_df['Line'].unique().tolist()

def cal_timetable(line):
    # Check if the line exists and is solved
    if line in lines and line in solved_lines:
        # First sailing time and headways
        d = x_df.loc[x_df['Line'] == line, 'Time'].iloc[0]
        headways = headway_df[f'h{line}'].dropna().tolist()

        times, locs, wharfs = [], [], []
        
        # Calculate timings for all sailings including first and subsequent ones
        for s in range(len(headways)+1):
            period = cal_h(s+1, d, line)  # Assuming cal_h is defined somewhere
            sailing_time = cal_time(period)
            
            # Add sailing time, location and wharf
            times.append(sailing_time)
            locs.append(Start_S[str(line)])
            wharfs.append(z_df[z_df['Task'] == str(line)]['Wharf'].iloc[0])

            # Intermediate Stop
            Intermediate_stop = line_df[line_df['Line_No'] == line]['I'].iloc[0]
            if pd.notna(Intermediate_stop):
                timetoI = int(line_df[line_df['Line_No'] == line]['Time_underway_to_I'].iloc[0])
                arrival_time_I = sailing_time + timedelta(minutes=timetoI)
                times.append(arrival_time_I)
                locs.append(Intermediate_stop)
                wharfs.append(z_df[z_df['Task'] == str(line)]['Wharf'].iloc[1])
            
            # Terminal Stop
            timetoT = int(line_df[line_df['Line_No'] == line]['Time_underway_to_T'].iloc[0])

            arrival_time_T = sailing_time + timedelta(minutes=timetoT)
            times.append(arrival_time_T)
            locs.append(End_S[str(line)])

            filtered_df = Zp_df[(Zp_df['Line'] == line) & (Zp_df['Time'] == period + cal_duration(timetoT))]

            # Check if filtered_df is not empty before accessing .iloc[0]
            if not filtered_df.empty:
                wharfs.append(filtered_df['Wharf'].iloc[0])
            else:
                print(f"No matching data found for line {line} and time {period + cal_duration(timetoT)}")
                # Handle the case when no match is found, e.g., append a placeholder or skip
                wharfs.append('No Wharf Found')  # or any other placeholder

            # wharfs.append(Zp_df[(Zp_df['Line'] == line) & (Zp_df['Time'] == period + timetoT // period_length + 1)]['Wharf'].iloc[0])

        # Format times and create the DataFrame
        formatted_times = [time.strftime('%H:%M') for time in times]
        timetable = pd.DataFrame({
            'Time': formatted_times,
            'Station': locs,
            'Wharf': wharfs
        })
        return timetable
    
    else:
        print('Line not exist or unsolved.')
        return None

cal_timetable(2)

,Time,Station,Wharf
0,06:00,Circular Quay,CQ1
1,06:27,Pyrmont Bay,PB1
2,07:05,Circular Quay,CQ4
3,06:20,Circular Quay,CQ1
4,06:47,Pyrmont Bay,PB1
5,07:25,Circular Quay,CQ2
6,06:40,Circular Quay,CQ1
7,07:07,Pyrmont Bay,PB1
8,07:45,Circular Quay,CQ4
9,07:00,Circular Quay,CQ1


In [20]:
Zp_df[Zp_df['Line'] == 2].sort_values(by = 'Time')

,Variable,Value,Line,Wharf,Time
595,"2,CQ4,20",1.0,2,CQ4,20
455,"2,CQ2,24",1.0,2,CQ2,24
603,"2,CQ4,28",1.0,2,CQ4,28
463,"2,CQ2,32",1.0,2,CQ2,32
611,"2,CQ4,36",1.0,2,CQ4,36
615,"2,CQ4,40",1.0,2,CQ4,40
691,"2,CQ5,44",1.0,2,CQ5,44
623,"2,CQ4,48",1.0,2,CQ4,48
483,"2,CQ2,52",1.0,2,CQ2,52
417,"2,CQ1,58",1.0,2,CQ1,58


In [25]:
Start_S['5']

'Circular Quay'

# Vessel itinerary

In [37]:
# Load data
# y_df = load_and_process_data('output_files/y_vjt_results.csv',['Vessel', 'Task', 'Start_Time'],{'Start_Time': int})
y_df = load_and_process_data(f'output_files/{file_prefix}_y_vjt_results.csv',['Vessel', 'Task', 'Start_Time'],{'Start_Time': int})

vi_df = y_df.copy()
# Extract and clean up Wharf details
vi_df['Start_Wharf'] = vi_df['Task'].apply(lambda x: x.split('_')[-1].strip())
vi_df['End_Wharf'] = vi_df['Start_Wharf']

# Lookup Start and End Stations based on Wharfs
vi_df['Start_Station'] = vi_df['Start_Wharf'].apply(lambda x: Start_S.get(x, 'Unknown Station'))
vi_df['End_Station'] = vi_df['Start_Wharf'].apply(lambda x: End_S.get(x, 'Unknown Station'))

# Update Task based on specific keywords or conditions
Bplus = ['CQ2', 'CQ4', 'CQ5', 'Bar1', 'Bar2', 'Chs1', 'Cab1', 'SOP1', 'Pm1', 'WB1', 'RB1', 'CI1', 'PB1']
Bc = ['cp_CQ1', 'cp_CQ2', 'cp_CQ3', 'cp_CQ4', 'cp_CQ5', 'cp_Bar1', 'cp_Bar2', 'cp_Bar4', 'cp_Bar5', 'cp_BSY1', 'cp_BSY2', 'cp_BSY3', 'cp_BSY4', 'cp_BSY5', 'cp_BSY6']
B = ['CQ1', 'CQ3', 'Bar4', 'Bar5', 'BSY1', 'BSY2', 'BSY3', 'BSY4', 'BSY5', 'BSY6', 'phi_CQ2', 'phi_CQ4', 'phi_CQ5', 'phi_Bar1', 'phi_Bar2', 'phi_Chs1', 'phi_Cab1', 'phi_SOP1', 'phi_Pm1', 'phi_WB1', 'phi_RB1', 'phi_CI1', 'phi_PB1']

vi_df['Task'] = vi_df['Task'].apply(
    lambda x: 'Waiting' if x in B else
              'Crew pause' if x in Bc else
              'Charging' if x in Bplus else
              f"{x}")

# Initial transformations for Start_Wharf and End_Wharf
vi_df['Start_Wharf'] = vi_df['Start_Wharf'].apply(start_wharf)
vi_df['End_Wharf'] = vi_df['Start_Wharf']  # Initially, set End_Wharf as Start_Wharf

vi_df['End_Wharf'] = vi_df.apply(end_wharf, axis=1)
vi_df['End_Time'] = vi_df.apply(end_time, axis=1)

# Reorganize DataFrame columns for final output
vi_df = vi_df[['Vessel', 'Task', 'Start_Station', 'Start_Wharf', 'Start_Time', 'End_Station', 'End_Wharf', 'End_Time']]

# Function to calculate vessel itinerary
def cal_itinerary(vessel):
    itinerary = vi_df[vi_df['Vessel'] == vessel].sort_values('Start_Time')
    # itinerary['Start_Time'] = itinerary['Start_Time'].apply(lambda x: cal_time(x).strftime('%H:%M'))
    itinerary.reset_index(inplace=True, drop=True)

    
    return itinerary


In [50]:
line_to_route_dict = {
    '1': 'F2 - Taronga Zoo',
    '2': 'F4 - Pyrmont Bay',
    '3': 'F5 - Neutral Bay',
    '4': 'F6 - Mosman',
    '5': 'F7 - Double Bay',
    '6': 'F8 - Cockatoo Island',
    '7': 'F9 - Rose Bay',
    '8': 'F9 - Watsons Bay',
    '9': 'F11 - Blackwattle Bay'}

testdf = cal_itinerary('V1').copy()
testdf['Task'] = testdf['Task'].replace(line_to_route_dict)
testdf

,Vessel,Task,Start_Station,Start_Wharf,Start_Time,End_Station,End_Wharf,End_Time
0,V1,Waiting,Barangaroo,Bar2,2,Barangaroo,Bar2,05:40
1,V1,Waiting,Circular Quay,CQ2,5,Circular Quay,CQ2,05:55
2,V1,Crew pause,Circular Quay,CQ1,6,Circular Quay,CQ1,06:04
3,V1,Waiting,Circular Quay,CQ5,8,Circular Quay,CQ5,06:10
4,V1,Crew pause,Circular Quay,CQ1,9,Circular Quay,CQ1,06:19
5,V1,Waiting,Circular Quay,CQ1,11,Circular Quay,CQ1,06:25
6,V1,F2 - Taronga Zoo,Circular Quay,CQ1,12,Circular Quay,CQ2,06:56
7,V1,Waiting,Circular Quay,CQ3,20,Circular Quay,CQ3,07:10
8,V1,F8 - Cockatoo Island,Circular Quay,CQ1,21,Circular Quay,CQ5,08:21
9,V1,Waiting,Barangaroo,Bar2,39,Barangaroo,Bar2,08:45


# Wharfs utilisations


In [34]:
# test_df = y_df[y_df['Start_Wharf'] == 'BWB1'].sort_values('Start_Time').copy()
# test_df['Start_Time'] = test_df['Start_Time'].apply(lambda x: cal_time(x).strftime('%H:%M'))
# test_df
y_df.head()# 对于有中间站的line来说start，wharf发生变动。y_df中储存的是一整条line task开始的时间而不是各个wharf的占用时间




,Variable,Value,Vessel,Task,Start_Time
1234,"10M,18,11",1.0,10M,18,11
1242,"10M,18,19",1.0,10M,18,19
1250,"10M,18,27",1.0,10M,18,27
1258,"10M,18,35",1.0,10M,18,35
1266,"10M,18,43",1.0,10M,18,43


In [ ]:
copydf = y_df.copy()
vessel_ls = y_df['Vessel'].unique().tolist()
line_ls = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19']
# int(row['Start_Time']) + x[1] for x in cal_delta(config, row['Task'], wharf)
# non_line_task_df = copydf[copydf['Task'].isnotin(line_ls)]
line_task_df = copydf[copydf['Task'].isin(line_ls)]
for vessel in vessel_ls:
    vessel_line_task_df = line_task_df[line_task_df]



AttributeError: 'Series' object has no attribute 'notin'

目的是根据y_df, z_df以及Zp_df创建一个新的dataframe，column为['v', 'w', 't_list'], t_list 是对船只v对特定码头w的占用时间

这个新的dataframe需要分为两部分获取，

第一个部分是非line的task，可以通过 y_df['Task']列不等于‘1’，‘2’，‘3’等等数字画的str来筛选，
筛选过后，在y_df中对每一个vessel进行loop，
对每个vessel来说，对筛选过后的仅包含一个vessel的y_vjt信息（他们在同一row）
v = row['Vessel'], j = row['Task'], w = row['Task'].split('-')[-1]， t = row['Start_Time']
t_list直接使用cal_delta（config, j, w）计算码头占用时间（local），将它转化成np.array之后可以直接+ t 得到码头占用时间（global），这个时间会是一个list。


第二个部分是对于line task来说。
相似的在y_df中，对vessel进行loop。
对筛选过后的仅包含一个vessel的y_vjt信息（他们在同一row）
v = row['Vessel'], j = int(row['Task']), w = Start_wharf[j]， t = row['Start_Time']
w是起始站，他的占用时间t_list为[time for time in range(t,t+Safety_buffer//period_length+1 +1)]Safety_buffer[line_df['Line_No] == j]['Safety_buffer'].iloc[0]

对于中间站，line_df[line_df['Line_No] == j]['I']如果不是pd.nan的话
w_i = z_df[z_df['Line'] == j]['Wharf'].iloc[1]
t_list中间站的占用时间通过cal_delta(config, j, w_i)计算，将它转化成np.array之后可以直接 + t 得到码头占用时间（global），这个时间是一个list。

对于终点站,
timetoT = int(line_df[line_df['Line_No'] == j]['Time_underway_to_T'].iloc[0])
arrival_T = t + timetoT//period_length + 1

在Zp_df中筛选，
w_t = Zp_df[(Zp_df['line'] == j)&(Zp_df['Time'] == arrival_T)]['Wharf'].unique().to_list()[0] 应该只有会有一个wharf出现
dw_T = int(line_df[line_df['Line_No'] == j]['dw_T'].iloc[0])
Safety_buffer = int(line_df[line_df['Line_No'] == j]['Safety_buffer'].iloc[0])
periods = (（dw_T + Safety_buffer）// period_length + 1)
t_list  = Zp_df[(Zp_df['line'] == j)&(Zp_df['Time'] >= arrival_T)&(Zp_df['Time'] <= arrival_T+ periods)]['Time'].unique().to_list(), 这个时间会是一个list。





In [37]:
line_df.head()


,Route_No,Line_No,O,I,T,dw_I,dw_T,Safety_buffer,First_sailing,Time_underway_to_I,Time_underway_to_T,Line_duration,Min_operating_speed,rj
0,F2,1,Circular Quay,NaN,Mosman,NaN,3,4,1900-01-01 06:48:00,NaN,15,18,13,0.041818
1,F2,2,Mosman,NaN,Circular Quay,NaN,3,5,1900-01-01 07:14:00,NaN,15,18,13,0.041818
2,F2,3,Circular Quay,NaN,Taronga Zoo,NaN,6,4,1900-01-01 08:50:00,NaN,12,18,13,0.041818
3,F2,4,Taronga Zoo,NaN,Circular Quay,NaN,5,5,1900-01-01 09:12:00,NaN,23,28,13,0.041818
4,F3,5,Circular Quay,Barangaroo,Parramatta,1.0,4,4,1900-01-01 07:07:00,11.0,86,90,20,0.000000


In [ ]:
cal_timetable(5)


Arrival time period: 39
Arrival time period: 44
Arrival time period: 49
Arrival time period: 54


,Time,Station,Wharf
0,07:10,Circular Quay,CQ1
1,07:21,Barangaroo,Bar1
2,08:36,Parramatta,Pm1
3,07:35,Circular Quay,CQ1
4,07:46,Barangaroo,Bar1
5,09:01,Parramatta,Pm1
6,08:00,Circular Quay,CQ1
7,08:11,Barangaroo,Bar1
8,09:26,Parramatta,Pm1
9,08:25,Circular Quay,CQ1


In [ ]:
# cal_timetable(18) --> Arrival time period
Arrival_T = [14,22,30,38,46,54,62]

# R_l = cal_Rl(j)  # Stations visited by the line


NameError: name 'occupied' is not defined

In [ ]:
Zp_df[Zp_df['Line'] == 18]

# first sailing start


,Variable,Value,Line,Wharf,Time
3817,"18,BWB1,2",1.0,18,BWB1,2
3818,"18,BWB1,3",1.0,18,BWB1,3
3819,"18,BWB1,4",1.0,18,BWB1,4
3829,"18,BWB1,14",1.0,18,BWB1,14
3830,"18,BWB1,15",1.0,18,BWB1,15
3837,"18,BWB1,22",1.0,18,BWB1,22
3838,"18,BWB1,23",1.0,18,BWB1,23
3845,"18,BWB1,30",1.0,18,BWB1,30
3846,"18,BWB1,31",1.0,18,BWB1,31
3853,"18,BWB1,38",1.0,18,BWB1,38


Every line l such that Z′ = 1. In this case, thanks to Eqs. (1h)-(1i), we know that there is exactly one
vessel v using wharf w as the last stop of line l, given by the right hand side of Eq. (1h).

columns:
Value	Vessel	Task	Start_Time	Start_Wharf	Occupied_time

In [ ]:
Zp_df['Task']  = Zp_df['line']
# 在zp_df 中time与